First, `pip install`

In [1]:
!pip install relbench

DEPRECATION: Loading egg at /Users/adarshjohny/Personal/Studies/Trento /Advanced ML/Project/ML/relbench/pyg_env/lib/python3.12/site-packages/pyg_lib-0.4.0-py3.12-macosx-11.0-arm64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached relbench-1.1.0-py3-none-any.whl.metadata (12 kB)
Using cached relbench-1.1.0-py3-none-any.whl (58 kB)


In [2]:
import relbench

relbench.__version__

'1.1.0'

To start we can check all of the databases currently available in RelBench by printing:

In [3]:
from relbench.datasets import get_dataset_names

get_dataset_names()

['rel-amazon',
 'rel-avito',
 'rel-event',
 'rel-f1',
 'rel-hm',
 'rel-stack',
 'rel-trial']

# Get dataset

Let's start with the F1 dataset since it's the smallest and is easy to work with. All it takes is one line!


In [4]:
from relbench.datasets import get_dataset

dataset = get_dataset(name="rel-f1", download=True)

Use `download=True` the first time you load a patricular dataset to automatically download the data from the RelBench server onto your machine.

Now we have loaded the database, let's start poking around to see what's inside.

# Val / Test cutoffs

We can check the val/test time cutoffs as follows:

In [5]:
dataset.val_timestamp, dataset.test_timestamp

(Timestamp('2005-01-01 00:00:00'), Timestamp('2010-01-01 00:00:00'))

This means that information upto 2005 can be used for training, and upto 2010 can be used for validation.

Note that it is a RelBench design choice to make the validation and test cutoffs a dataset property, _not_ a task-specific property. In other words, all tasks for a given database use the same time splits.


# Load database

Next we check out the database itself...

In [6]:
db = dataset.get_db()

Loading Database object from /Users/adarshjohny/Library/Caches/relbench/rel-f1/db...
Done in 0.08 seconds.


*This* returns a RelBench `Database` object. By default, the rows with timestamp > test_timestamp are excluded to prevent accidental test set leakage.
The complete database can be loaded with `database.get_db(upto_test_timestamp=False)`.

With this we can double check the full timespan of the database:

In [7]:
db.min_timestamp, db.max_timestamp

(Timestamp('1950-05-13 00:00:00'), Timestamp('2009-11-01 11:00:00'))

1950 is the first season for F1! So we have data for the full history of F1. Note that the `max_timestamp` is the same as `test_timestamp`.

Next let's check out the tables in the database.

More info on the schemas for F1 and all other datasets can be found at https://relbench.stanford.edu/.

We have the following tables:

In [8]:
db.table_dict.keys()

dict_keys(['qualifying', 'drivers', 'results', 'standings', 'races', 'constructors', 'constructor_results', 'circuits', 'constructor_standings'])

That's 9 tables total! Let's look more closely at one of them.

In [9]:
table = db.table_dict["drivers"]
table

Table(df=
     driverId        driverRef code  forename     surname        dob  \
0           0         hamilton  HAM     Lewis    Hamilton 1985-01-07   
1           1         heidfeld  HEI      Nick    Heidfeld 1977-05-10   
2           2          rosberg  ROS      Nico     Rosberg 1985-06-27   
3           3           alonso  ALO  Fernando      Alonso 1981-07-29   
4           4       kovalainen  KOV    Heikki  Kovalainen 1981-10-19   
..        ...              ...  ...       ...         ...        ...   
852       852  mick_schumacher  MSC      Mick  Schumacher 1999-03-22   
853       853             zhou  ZHO    Guanyu        Zhou 1999-05-30   
854       854         de_vries  DEV      Nyck    de Vries 1995-02-06   
855       855          piastri  PIA     Oscar     Piastri 2001-04-06   
856       856         sargeant  SAR     Logan    Sargeant 2000-12-31   

    nationality  
0       British  
1        German  
2        German  
3       Spanish  
4       Finnish  
..          ...  

The `drivers` table stores information on all F1 drivers that ever competed in a race. Note that the table comes with multiple bits of information:
- The table itself, `table.df` which is simply a Pandas DataFrame.
- The primary key column, `table.pkey_col`, which indicates that the `driverId` column holds the primary key for this particular table in the database.
- The primary time column, `table.time_col` which, if the entity is an event, records the time an event happened. In the case of drivers, they are non-temporal entities, so `table.time_col=None`.
- The other tables that foreign keys points to `table.fkey_col_to_pkey_table`. If the table has any foreign key columns, then this dict indicates which table we foreign key corresponds to. Again in the case of drivers this is not applicable.

We can start to explore the data a little, e.g., check out the oldest and youngest ever F1 drivers, spanning 3 centuries!

In [10]:
table.df.iloc[table.df["dob"].idxmax()]

driverId                       855
driverRef                  piastri
code                           PIA
forename                     Oscar
surname                    Piastri
dob            2001-04-06 00:00:00
nationality             Australian
Name: 855, dtype: object

In [11]:
table.df.iloc[table.df["dob"].idxmin()]

driverId                       741
driverRef                etancelin
code                            \N
forename                  Philippe
surname                  Étancelin
dob            1896-12-28 00:00:00
nationality                 French
Name: 741, dtype: object

Going back to the `table.time_col` and `table.fkey_col_to_pkey_table`, the `results` table contains a non-trivial example.

In [12]:
table = db.table_dict["results"]
table.df

,resultId,raceId,driverId,constructorId,number,grid,position,positionOrder,points,laps,milliseconds,fastestLap,rank,statusId,date
0,0,0,660,152,18.0,21,11.0,11,0.0,64,NaN,NaN,NaN,16,1950-05-13 00:00:00
1,1,0,790,149,8.0,12,NaN,21,0.0,2,NaN,NaN,NaN,126,1950-05-13 00:00:00
2,2,0,579,49,1.0,3,NaN,12,0.0,62,NaN,NaN,NaN,44,1950-05-13 00:00:00
3,3,0,661,149,9.0,10,NaN,20,0.0,5,NaN,NaN,NaN,6,1950-05-13 00:00:00
4,4,0,789,152,17.0,7,NaN,19,0.0,8,NaN,NaN,NaN,51,1950-05-13 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20318,20318,819,1,1,6.0,8,5.0,5,4.0,55,5669667.0,54.0,7.0,1,2009-11-01 11:00:00
20319,20319,819,21,22,23.0,4,4.0,4,5.0,55,5666149.0,54.0,4.0,1,2009-11-01 11:00:00
20320,20320,819,17,22,22.0,5,3.0,3,6.0,55,5661881.0,49.0,6.0,1,2009-11-01 11:00:00
20321,20321,819,16,8,14.0,3,2.0,2,8.0,55,5661271.0,14.0,5.0,1,2009-11-01 11:00:00


Here we start to notice certain data artifacts that might be good to keep in mind for later when doing ML modeling. For instance, the `milliseconds` and `fastestLap` columns seem to only have been collected for more recent races, with `NaN` features for earlier races.

# Loading a task

Each RelBench dataset comes with multiple pre-defined predictive tasks. For any given RelBench dataset, you can check all the associated tasks with:

In [13]:
from relbench.tasks import get_task_names, get_task

get_task_names("rel-f1")

['driver-position', 'driver-dnf', 'driver-top3']

Check out https://relbench.stanford.edu/ for detailed descriptions of what each task is. As an example, let's use `driver-top3` where the task is, for a given driver and a given timestamp, to predict whether that driver will finish in the top 3 in some race in the next 30 days.

The task itself is instantiated by calling:

In [14]:
task = get_task("rel-f1", "driver-top3", download=True)

Next we load the train / val / test labels. Each task table contains triples (timestamp, Id, label) indicating the entity the label is associated to, the timepoint at which the prediction is made, an the label itself. The task table also indicates which database table it is "attached" to - in this case the the `drivers` table.

In [15]:
task.get_table("train")

Table(df=
           date  driverId  qualifying
0    2004-08-04        12           0
1    2004-08-04        20           0
2    2004-07-05        10           0
3    2004-07-05        47           0
4    2004-06-05        31           0
...         ...       ...         ...
1348 1994-03-30        80           0
1349 1994-03-30        48           0
1350 1994-03-30        77           0
1351 1994-02-28        43           0
1352 1994-02-28        56           0

[1353 rows x 3 columns],
  fkey_col_to_pkey_table={'driverId': 'drivers'},
  pkey_col=None,
  time_col=date)

The test table is handled differently, with the labels being hidden by default to prevent accidental test set leakage.

In [16]:
task.get_table("test")

Table(df=
          date  driverId
0   2013-03-16       814
1   2012-11-16         9
2   2012-11-16        17
3   2012-10-17         0
4   2012-09-17       816
..         ...       ...
721 2010-07-30        14
722 2010-06-30       154
723 2010-06-30        14
724 2010-05-01        14
725 2010-05-01       154

[726 rows x 2 columns],
  fkey_col_to_pkey_table={'driverId': 'drivers'},
  pkey_col=None,
  time_col=date)

If strictly needed, test labels can be retrieved by calling:

In [17]:
task.get_table("test", mask_input_cols=False)

Table(df=
          date  driverId  qualifying
0   2013-03-16       814           0
1   2012-11-16         9           0
2   2012-11-16        17           1
3   2012-10-17         0           1
4   2012-09-17       816           0
..         ...       ...         ...
721 2010-07-30        14           0
722 2010-06-30       154           0
723 2010-06-30        14           0
724 2010-05-01        14           0
725 2010-05-01       154           0

[726 rows x 3 columns],
  fkey_col_to_pkey_table={'driverId': 'drivers'},
  pkey_col=None,
  time_col=date)